In [1]:
import numpy as np
# import cupy as cp
# print(cp.__version__)
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys

In [2]:
def print_branches(skeleton_cleaned):
    skel = Skeleton(skeleton_cleaned)
    branch_data = summarize(skel)

    branch_type_0 = branch_data[branch_data['branch-type'] == 0]
    branch_type_1 = branch_data[branch_data['branch-type'] == 1]
    branch_type_2 = branch_data[branch_data['branch-type'] == 2]
    branch_type_3 = branch_data[branch_data['branch-type'] == 3]

    print(f"Number of branch type 0: {len(branch_type_0)}")
    print(f"Number of branch type 1: {len(branch_type_1)}")
    print(f"Number of branch type 2: {len(branch_type_2)}")
    print(f"Number of branch type 3: {len(branch_type_3)}")

In [3]:
# binary_volume = tiff.imread("/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume.tif")
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/skeleleton_clean_x20_th5.tif")
# Get the size of the volume
print(f'Size of the volume is: {binary_volume.shape}')

Size of the volume is: (200, 200, 200)


In [4]:
print_branches(binary_volume)

Number of branch type 0: 11
Number of branch type 1: 27
Number of branch type 2: 17
Number of branch type 3: 1


In [5]:
skel = Skeleton(binary_volume)
branch_data = summarize(skel)

In [6]:
import cc3d
import numpy as np

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(N)

22


In [7]:
# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


In [8]:
def bresenham_line_3d(p1, p2):
    # Implementa el algoritmo de Bresenham en 3D para trazar una línea entre dos puntos
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    points = []

    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    sz = 1 if z1 < z2 else -1
    err = dx - dy - dz

    while True:
        points.append((x1, y1, z1))
        if (x1, y1, z1) == (x2, y2, z2):
            break
        e2 = err * 2
        if e2 > -dy - dz:
            err -= dy
            x1 += sx
        if e2 < dx - dz:
            err += dx
            y1 += sy
        if e2 < dx - dy:
            err += dz
            z1 += sz
    return points


In [15]:
import numpy as np
from scipy.spatial.distance import cdist

# Paso 1: Calcular centroides de los blobs en 3D
def calculate_centroid(blob_labels, labels_out):
    centroids = {}
    for blob in np.unique(blob_labels):
        # Encuentra las coordenadas de todos los píxeles de ese blob
        coords = np.array(np.where(labels_out == blob)).T
        # Calcula el centroide como el promedio de las coordenadas
        centroid = np.mean(coords, axis=0)
        centroids[blob] = centroid
    return centroids

# Paso 2: Encontrar los blobs más cercanos en 3D
def find_closest_blobs(centroids):
    blobs = list(centroids.keys())
    distances = cdist([centroids[blob] for blob in blobs], [centroids[blob] for blob in blobs])
    np.fill_diagonal(distances, np.inf)  # Evitar que los blobs se conecten consigo mismos
    closest_pairs = np.unravel_index(np.argmin(distances), distances.shape)
    return blobs[closest_pairs[0]], blobs[closest_pairs[1]]

# Paso 3: Conectar los blobs más cercanos con una línea recta 3D
def connect_blobs_3d(labels_out, pixel_blob_mapping, skeleton_cleaned):
    # Calcular centroides de todos los blobs
    skel = np.copy(skeleton_cleaned)
    centroids = calculate_centroid(labels_out, labels_out)

    # Encontrar el par de blobs más cercanos
    blob1, blob2 = find_closest_blobs(centroids)
    print(f"Connecting blobs {blob1} and {blob2}")

    # Encuentra un punto en cada blob
    coords_blob1 = np.array(np.where(labels_out == blob1)).T
    coords_blob2 = np.array(np.where(labels_out == blob2)).T

    # Seleccionar el primer píxel de cada blob
    p1 = coords_blob1[0]
    p2 = coords_blob2[0]

    # Calcular la línea de Bresenham entre los puntos (en 3D)
    line_points = bresenham_line_3d(p1, p2)

    # Conectar los blobs en el mapeo de píxeles
    for point in line_points:
        pixel_blob_mapping[tuple(point)] = blob1  # O puedes usar una mezcla de ambos blobs
        skel[tuple(point)] = 1
    
    return skel


In [17]:
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    skel = connect_blobs_3d(labels_out, pixel_blob_mapping, skel)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping
    
    labels_out, N = cc3d.connected_components(skel, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)
    
    if N == 1:
        print("All blobs are connected!")
        break

MemoryError: Unable to allocate 183. MiB for an array with shape (3, 7999723) and data type int64

In [ ]:
# blob_mapping_connected = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume)
tiff.imwrite('blob_mapping_connected.tif', skel)


23


In [11]:
# Assuming blob_mapping_connected is a 3D NumPy array
# Step 1: Convert to binary
binary_blob = (blob_mapping_connected > 0)

# Step 2: Skeletonize
skeleton_connected = skeletonize(binary_blob)
print_branches(skeleton_connected)

Number of branch type 0: 13
Number of branch type 1: 26
Number of branch type 2: 8
Number of branch type 3: 0


In [23]:
output_path_cleaned = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/skeleton_connected.tif'
tiff.imwrite(output_path_cleaned, skeleton_connected)